In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products__train = pd.read_csv("order_products__train.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")

## Exploratory Analysis


In [7]:
pip install ollama

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import asyncio
import pandas as pd
from ollama import AsyncClient

# 1. Load products.csv
products = pd.read_csv("products.csv")

# 2. Initialize Ollama client
ollama = AsyncClient(host="http://localhost:11434")

async def fetch_price(product_name: str) -> float:
    """
    Send product_name to Ollama and parse back a dollar price.
    Assumes Ollama returns something like "2.99".
    """
    prompt = (
        f"Suggest a realistic U.S. grocery retail price (in dollars) for the product:\n\n"
        f"\"{product_name}\"\n\n"
        f"Return only a number with two decimal places, e.g. 3.49"
    )
    try:
        resp = await ollama.chat(
            model="llama2", 
            messages=[{"role":"user","content":prompt}]
        )
        text = resp["message"]["content"].strip()
        # parse float from response
        price = float(text)
        return round(price, 2)
    except Exception as e:
        print(f"Failed to get price for '{product_name}': {e}")
        return None

async def main():
    # Create a list to hold fetched prices
    prices = []
    
    # Iterate sequentially (could batch or parallelize if desired)
    for name in products["product_name"]:
        price = await fetch_price(name)
        prices.append(price)
    
    # Assign back into DataFrame
    products["price"] = prices
    
    # Save enriched CSV
    out_file = "products_with_price.csv"
    products.to_csv(out_file, index=False)
    print(f"Saved enriched file to {out_file}")

if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop